In [23]:
from os import listdir, getcwd
from os.path import isfile, join
import pandas as pd

pathPrefix = "../data/files/"
dataPath = getcwd() + "/" + pathPrefix
print(dataPath)

files = [f for f in listdir(dataPath) if isfile(join(dataPath, f))]

/home/juliano/projects/Bus-Trips-In-Brazil/src/../data/files/


In [24]:
import charset_normalizer

encodings = {}

for file in files:
    with open(pathPrefix + file, 'rb') as rawdata:
        encodings[file] = charset_normalizer.detect(rawdata.read(10000))
        
for file in encodings:
    print(file, encodings[file])
    print()

viagem_regular_10_2022.csv {'encoding': 'Windows-1252', 'language': 'Portuguese', 'confidence': 1.0}

viagem_regular_08_2024.csv {'encoding': 'windows-1250', 'language': 'Portuguese', 'confidence': 1.0}

viagem_regular_12_2021.csv {'encoding': 'windows-1250', 'language': 'Portuguese', 'confidence': 1.0}

viagem_regular_07_2020.csv {'encoding': 'windows-1250', 'language': 'Portuguese', 'confidence': 1.0}

viagem_regular_12_2020.csv {'encoding': 'windows-1250', 'language': 'Portuguese', 'confidence': 1.0}

viagem_regular_05_2020.csv {'encoding': 'windows-1250', 'language': 'Dutch', 'confidence': 1.0}

viagem_regular_06_2019.csv {'encoding': 'Windows-1252', 'language': 'Portuguese', 'confidence': 1.0}

viagem_regular_02_2019.csv {'encoding': 'windows-1250', 'language': 'Portuguese', 'confidence': 1.0}

viagem_regular_09_2019.csv {'encoding': 'Windows-1252', 'language': 'Portuguese', 'confidence': 1.0}

viagem_regular_11_2022.csv {'encoding': 'windows-1250', 'language': 'Polish', 'confiden

In [25]:
import numpy as np

colTypes = {'codigo_viagem':'str', 
            'cnpj':'str',
            'placa':'str',
            'nu_linha':'str',            
            'tipo_viagem':'str',
            'data_viagem_programada':'str',
            'hora_viagem_programada':'str',
            'data_inicio_viagem':'str',
            'data_fim_viagem':'str',
            'sentido_linha':'str',
            'latitude':np.float64,
            'longitude':np.float64,
            'pdop':np.float64,
            'numero_imei':'str',
            'in_transbordo':'str',
            'codigo_viagem_transbordo':'str'
           }

firstFile = files[0]
encoding = encodings[firstFile]

df = pd.read_csv(pathPrefix + firstFile, encoding=encoding['encoding'], delimiter=';', header=0, dtype=colTypes)
df.head()

,codigo_viagem,cnpj,placa,nu_linha,tipo_viagem,data_viagem_programada,hora_viagem_programada,data_inicio_viagem,data_fim_viagem,sentido_linha,latitude,longitude,pdop,numero_imei,in_transbordo,codigo_viagem_transbordo
0,5105552,78586674000107,SDS0J21,19008551,0,13-10-2022,18:30:00,13-10-2022 17:35:52,14-10-2022 11:09:00,0,-22.228829,-54.772912,0.0,7223b45b6a79c34,NÃO,NaN
1,5105563,78586674000107,SDQ9G47,09039031,0,13-10-2022,18:45:00,13-10-2022 18:43:31,13-10-2022 19:52:31,1,-22.979963,-49.902584,0.0,353028105417459,NÃO,NaN
2,5105564,78586674000107,BCY9I92,09039031,0,13-10-2022,18:50:00,13-10-2022 17:58:29,13-10-2022 19:50:04,0,-23.045506,-50.230369,0.0,353028105432078,NÃO,NaN
3,5105566,78586674000107,BCF5549,09014451,0,13-10-2022,19:00:00,13-10-2022 18:21:21,14-10-2022 08:40:15,1,-23.933118,-46.334373,0.0,f9e7972384824e9,NÃO,NaN
4,5105574,78586674000107,RHX7E78,09014451,0,13-10-2022,19:00:00,13-10-2022 16:54:36,14-10-2022 07:55:37,0,-22.980955,-49.902951,0.0,39199bf35f3e3a8,NÃO,NaN


In [26]:
df.dtypes

codigo_viagem                object
cnpj                         object
placa                        object
nu_linha                     object
tipo_viagem                  object
data_viagem_programada       object
hora_viagem_programada       object
data_inicio_viagem           object
data_fim_viagem              object
sentido_linha                object
latitude                    float64
longitude                   float64
pdop                        float64
numero_imei                  object
in_transbordo                object
codigo_viagem_transbordo     object
dtype: object

In [27]:
codeSizes = df["codigo_viagem"].str.len()
print(codeSizes.groupby(codeSizes).value_counts())

cnpjSizes = df["cnpj"].str.len()
print(cnpjSizes.groupby(cnpjSizes).value_counts())

lineSizes = df["nu_linha"].str.len()
print(lineSizes.groupby(lineSizes).value_counts())

plateSizes = df["placa"].str.len()
print(plateSizes.groupby(plateSizes).value_counts())

imeiSizes = df["numero_imei"].str.len()
print(imeiSizes.groupby(imeiSizes).value_counts())

codigo_viagem  codigo_viagem
7              7                100251
Name: count, dtype: int64
cnpj  cnpj
14    14      100251
Name: count, dtype: int64
nu_linha  nu_linha
8         8           100251
Name: count, dtype: int64
placa  placa
7      7        100251
Name: count, dtype: int64
numero_imei  numero_imei
15           15             100251
Name: count, dtype: int64


In [72]:
import psycopg2
import os

conn = psycopg2.connect(host="localhost", port=5432, dbname="ANTT", user=os.environ["PGDBUSER"], password=os.environ["PGDBPASSWORD"])

In [73]:
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS RegularTrips (
    id VARCHAR(10) PRIMARY KEY,
    cnpj VARCHAR(20),
    plate VARCHAR(10),
    line VARCHAR(10),
    tripType VARCHAR(20),
    expectedTripDate DATE,
    expectedTripHour TIME,
    tripStartDateTime TIMESTAMP,
    tripEndDateTime TIMESTAMP,
    lineDirection BOOL,
    latitude REAL,
    longitude REAL,
    pdop REAL,
    imei VARCHAR(20),
    transbordo BOOL,
    transbordoCode VARCHAR(20)
    );
""")
conn.commit()

In [67]:
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS RegularTripsTests (
    id VARCHAR(10) PRIMARY KEY,
    cnpj VARCHAR(20),
    plate VARCHAR(10),
    line VARCHAR(10),
    tripType VARCHAR(20),
    expectedTripDate DATE,
    expectedTripHour TIME,
    tripStartDateTime TIMESTAMP,
    tripEndDateTime TIMESTAMP,
    lineDirection BOOL,
    latitude REAL,
    longitude REAL,
    pdop REAL,
    imei VARCHAR(20),
    transbordo BOOL,
    transbordoCode VARCHAR(20)
    );
""")
conn.commit()

In [74]:
conn.close()

In [84]:
from datetime import date, time, datetime

def SaveBatchToDB(conn, tableName, data):
    cur = conn.cursor()    
    cur.executemany(
        f"""
        INSERT INTO
            {tableName}
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, 
        data
    )
    conn.commit()

def PreprocessBatch(data, headerMap, conn, tableName):
    preprocessedData = []
    
    for entry in data:
        preprocessedData.append(PreprocessEntry(entry, headerMap))

    print("Succesfully preprocessed current data.")
    SaveBatchToDB(conn, tableName, preprocessedData)

def ProcessFloatNotation(num):
    '''
        Filters string so it can be converted to float
        "1.000.001,23" -> "1000001.23"
        "1,000,001.23" -> "1000001.23"
    '''
    decimalIdx = -1
    for i in range(len(num) - 1, -1, -1):
        if num[i] == '.' or num[i] == ',':
            decimalIdx = i
            break

    result = ""
    for i in range(len(num)):
        if i == decimalIdx:
            result += '.'
        elif num[i].isnumeric() or num[i] == '-':
            result += num[i]
            
    if decimalIdx == -1:
        result += ".0"
    return result

def ProcessDateTime(entry, mode):
    if not len(entry):
        return None
    
    if mode == "date":
        d, m, Y = map(int, entry.split("-"))
        return date(day=d, month=m, year=Y)
    if mode == "time":
        return time.fromisoformat(entry)
    if mode == "datetime":
        dateSplit, timeSplit = entry.split(" ")
        d, m, Y = map(int, dateSplit.split("-"))
        
        dateParsed = date(day=d, month=m, year=Y)
        timeParsed = time.fromisoformat(timeSplit)
        
        return datetime.combine(dateParsed, timeParsed)
    raise Exception(f"Invalid mode was given in ProcessDateTime: {mode}")

def PreprocessEntry(entry, headerMap):
    ''' 
        Process fields to standardize formatting and convert types
    '''

    entry[headerMap['codigo_viagem']] = "".join(n for n in entry[headerMap['codigo_viagem']] if n.isnumeric())
    entry[headerMap['cnpj']] = "".join(n for n in entry[headerMap['cnpj']] if n.isnumeric())
    entry[headerMap['placa']] = "".join(n for n in entry[headerMap['placa']] if n.isalnum())
    entry[headerMap['nu_linha']] = "".join(n for n in entry[headerMap['nu_linha']] if n.isalnum())

    entry[headerMap['data_viagem_programada']] = ProcessDateTime(entry[headerMap['data_viagem_programada']], "date")
    entry[headerMap['hora_viagem_programada']] = ProcessDateTime(entry[headerMap['hora_viagem_programada']], "time")
    entry[headerMap['data_inicio_viagem']] = ProcessDateTime(entry[headerMap['data_inicio_viagem']], "datetime")
    entry[headerMap['data_fim_viagem']] = ProcessDateTime(entry[headerMap['data_fim_viagem']], "datetime")   

    tripType = entry[headerMap['tipo_viagem']]
    if len(tripType) > 20:
        raise Exception(f"Invalid value for 'tipo_viagem': {tripType}. Values should be no longer than 20 characters.")        

    lineDirection = entry[headerMap['sentido_linha']]
    if lineDirection != '0' and lineDirection != '1':
        raise Exception(f"Invalid value for 'sentido_linha': {entry[headerMap['sentido_linha']]}. Values should be either 0 or 1.")

    entry[headerMap['latitude']] = float(ProcessFloatNotation(entry[headerMap['latitude']]))
    entry[headerMap['longitude']] = float(ProcessFloatNotation(entry[headerMap['longitude']]))
    entry[headerMap['pdop']] = float(ProcessFloatNotation(entry[headerMap['pdop']]))

    transbordoValue = entry[headerMap['in_transbordo']].lower()    
    if transbordoValue == 'não':
        entry[headerMap['in_transbordo']] = False
    elif transbordoValue == 'sim':
        entry[headerMap['in_transbordo']] = True
    else:
        raise Exception(f"Invalid value for 'in_transbordo': {transbordoValue}. Values should be either 'não' or 'sim'.")

    return entry

In [75]:
processedFiles = set([])

# Assumes batchsize does not change between executions
failedFiles = {}

In [82]:
import csv
from itertools import batched

conn = psycopg2.connect(host="localhost", port=5432, dbname="ANTT", user=os.environ["PGDBUSER"], password=os.environ["PGDBPASSWORD"])
tableName = "RegularTrips"
encoding = "windows-1252"

for i in range(len(files)):
    nextFileIndex = i + 1
    filePath = pathPrefix + files[i]
    
    if filePath in processedFiles:
        continue
    
    print(f"Current file being processed {filePath} {i}")    
    batchSize = 10000

    with open(filePath, encoding=encoding) as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        header = next(reader)
        headerMap = dict(zip(header, range(len(header))))

        for i, batch in enumerate(batched(reader, batchSize)):
            if filePath in failedFiles and i * batchSize < failedFiles[filePath]:
                continue            
            try:
                PreprocessBatch(batch, headerMap, conn, tableName)
            except Exception as e:
                failedFiles[filePath] = i*batchSize
                print(e)                
                raise Exception(f"Failed to fully process and save file: {filePath}. Current batch: {i*batchSize}")
                
            print(f"Succesfully preprocessed and saved batch from line {i*batchSize} to {(i+1)*batchSize-1} of file {filePath}")

    failedFiles.pop(filePath, None) # Remove file from failed files if it was present
    processedFiles.add(filePath)

conn.close()

Current file being processed ../data/files/viagem_regular_07_2020.csv 3
Succesfully preprocessed current data.
Succesfully preprocessed and saved batch from line 0 to 9999 of file ../data/files/viagem_regular_07_2020.csv
Succesfully preprocessed current data.
Succesfully preprocessed and saved batch from line 10000 to 19999 of file ../data/files/viagem_regular_07_2020.csv
Succesfully preprocessed current data.
Succesfully preprocessed and saved batch from line 20000 to 29999 of file ../data/files/viagem_regular_07_2020.csv
Current file being processed ../data/files/viagem_regular_12_2020.csv 4
Succesfully preprocessed current data.
Succesfully preprocessed and saved batch from line 0 to 9999 of file ../data/files/viagem_regular_12_2020.csv
Succesfully preprocessed current data.
Succesfully preprocessed and saved batch from line 10000 to 19999 of file ../data/files/viagem_regular_12_2020.csv
Succesfully preprocessed current data.
Succesfully preprocessed and saved batch from line 20000 

In [83]:
for file in processedFiles:
    print(file)

for file in failedFiles:
    print(file, failedFiles[file])

../data/files/viagem_regular_07_2023.csv
../data/files/viagem_regular_10_2020.csv
../data/files/viagem_regular_12_2019.csv
../data/files/viagem_regular_08_2023.csv
../data/files/viagem_regular_09_2020.csv
../data/files/viagem_regular_02_2021.csv
../data/files/viagem_regular_03_2020.csv
../data/files/viagem_regular_11_2022.csv
../data/files/viagem_regular_02_2022.csv
../data/files/viagem_regular_05_2019.csv
../data/files/viagem_regular_02_2020.csv
../data/files/viagem_regular_04_2022.csv
../data/files/viagem_regular_01_2021.csv
../data/files/viagem_regular_03_2019.csv
../data/files/viagem_regular_07_2022.csv
../data/files/viagem_regular_06_2019.csv
../data/files/viagem_regular_06_2023.csv
../data/files/viagem_regular_12_2023.csv
../data/files/viagem_regular_02_2023.csv
../data/files/viagem_regular_11_2021.csv
../data/files/viagem_regular_12_2024.csv
../data/files/viagem_regular_08_2021.csv
../data/files/viagem_regular_10_2023.csv
../data/files/viagem_regular_12_2020.csv
../data/files/vi

In [88]:
count = 0
totalEntries = 0
idsWithIssue = []

for file in files:
    filePath = pathPrefix + files[i]
    
    batchSize = 10000

    with open(filePath, encoding=encoding) as csvfile:
        reader = csv.reader(csvfile, delimiter=';')
        header = next(reader)
        headerMap = dict(zip(header, range(len(header))))

        for row in reader:
            totalEntries += 1
            latitude = float(ProcessFloatNotation(row[headerMap['latitude']]))
            longitude = float(ProcessFloatNotation(row[headerMap['longitude']]))

            if latitude > 0:
                count += 1
                idsWithIssue.append(row[headerMap['codigo_viagem']])
            if longitude > 0:
                count += 1                
                idsWithIssue.append(row[headerMap['codigo_viagem']])

print(f"Total latitude or longitude values above zero: {count}")
print(f"Total entries: {totalEntries}")
print(idsWithIssue)
                

Total latitude or longitude values above zero: 19080
Total entries: 5572872
['1347992', '1314240', '1323424', '1323423', '1343045', '1343046', '1314249', '1314269', '1347990', '1347993', '1323428', '1323427', '1314298', '1314314', '1347996', '1314342', '1343053', '1343054', '1343057', '1343058', '1313621', '1314351', '1343059', '1343060', '1350306', '1350335', '1349882', '1358729', '1350684', '1349884', '1358732', '1358731', '1350343', '1350366', '1349890', '1358740', '1358739', '1350398', '1349891', '1350414', '1350443', '1349893', '1350687', '1358742', '1349894', '1358743', '1358744', '1350450', '1350471', '1369046', '1369040', '1358749', '1350689', '1358752', '1358753', '1350505', '1350525', '1369047', '1350553', '1350690', '1349902', '1350567', '1350582', '1349903', '1369041', '1369048', '1349904', '1358758', '1358757', '1350691', '1349905', '1350617', '1350629', '1369045', '1350656', '1358760', '1377852', '1377851', '1349907', '1350662', '1373952', '1373976', '1379072', '1377854',

In [93]:
# Executed this to fix latitude values that had their sign incorrectly signed

# conn = psycopg2.connect(host="localhost", port=5432, dbname="ANTT", user=os.environ["PGDBUSER"], password=os.environ["PGDBPASSWORD"])
# cur = conn.cursor()

# cur.execute("""
#     UPDATE REGULARTRIPS
#     SET latitude = -latitude
#     WHERE id = ANY(%s);
# """, (idsWithIssue, ))

# conn.commit()
# conn.close()